In [12]:
import torch 
import gpytorch
import numpy as np
import sys 
import os
sys.path.append(os.path.abspath('..'))
from plotting import plot_train_test_split, plot_grid
from data.utils import *
from data.generator import PVDataGenerator
from data.loader import PVDataLoader
from models.multitask import MultitaskGPModel

In [13]:
# set seed for reproducibility
SEED = 42
torch.manual_seed(SEED)
np.random.seed(SEED)

# data parameters
DAY_INIT = 0
DAY_MIN = 8
DAY_MAX = 16
N_DAYS = 100
N_DAYS_TRAIN = 5
MINUTE_INTERVAL = 5
DAILY_DATA_POINTS = (DAY_MAX - DAY_MIN) * 60 / MINUTE_INTERVAL
N_HOURS_PRED = 2
N_SYSTEMS = 55
RADIUS = 0.35
CIRCLE_COORDS = (55, -1.5)
# create a tuple of 4 coordinates that form a polygon in the uk

POLY_COORDS = ((50, -6), (50.5, 1.9), (57.6, -5.5), (58, 1.9))

In [14]:
generator = PVDataGenerator(n_days=N_DAYS,
                    day_init=DAY_INIT,
                    n_systems=N_SYSTEMS,
                    coords=CIRCLE_COORDS,
                    minute_interval=MINUTE_INTERVAL,
                    day_min=DAY_MIN,
                    day_max=DAY_MAX,
                    folder_name='pv_data',
                    file_name_pv='pv_data_clean.csv',
                    file_name_location='location_data_clean.csv',
                    distance_method='circle',
                    drop_nan=False)

X, y = generator.get_time_series()
num_latent = 5
num_tasks = y.shape[1]

==> Loading data
==> Loaded data in: 0 m : 8 sec

==> Loading data
==> Loaded data in: 0 m : 0 sec



In [18]:
x_train, y_train, x_test, y_test = train_test_split(X, y, n_hours=N_HOURS_PRED)

In [19]:
from kernels.quasi_periodic import generate_quasi_periodic
from likelihoods.beta import MultitaskBetaLikelihood

In [25]:
mean = gpytorch.means.ConstantMean(batch_shape=torch.Size([num_latent]))
kernel = generate_quasi_periodic(num_latent=num_latent)
beta_likelihood = MultitaskBetaLikelihood(num_tasks=num_tasks, scale=20)
gaussian_likelihood = gpytorch.likelihoods.MultitaskGaussianLikelihood(num_tasks=num_tasks)

In [29]:
beta_model = MultitaskGPModel(x_train, y_train, 
                            likelihood=beta_likelihood,
                            mean_module=mean,
                            covar_module=kernel,
                            num_latents=num_latent)

gaussian_model = MultitaskGPModel(x_train, y_train,
                            likelihood=gaussian_likelihood,
                            mean_module=mean,
                            covar_module=kernel,
                            num_latents=num_latent)

In [ ]:
beta_model.fit(n_iter=500, lr=0.1, verbose=True)
gaussian_model.fit(n_iter=500, lr=0.1, verbose=True)